# Echo-A 1B5 (Memory Finetune 1)
This continues off from `Echo-A-1B5-basemodel.ipynb` to perform the full memory finetune & testing process

This performs the training with limited masking, and ensure the model learns the memorising instructions, and is able to perform some limited memorisation task. This is before additional finetune (finetune 2) which would focus only on the completion masking loss (and not the instruct / prompt)

In all cases, the input tokens is always masked.

> This project assumes you have the rwkv-infctx conda env setup, and you are executing in that environment - see the main README.md for the conda env setup steps

## Optional: Download the pretrained model
(if you want to skip the the basemodel train + instruct tune)


In [1]:
# Init required dirs
!mkdir -p ../../../model/
!mkdir -p ../../../datapath/
!mkdir -p ../../../checkpoint/

# Download the Stage2.pth file
!rm -rf ../../../model/Echo-A-1B5-Stage2.pth
!cd ../../../model/ && wget https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Stage2.pth
!ls -alh ../../../model/Echo-A-1B5-Stage2.pth

--2023-07-12 10:48:17--  https://huggingface.co/picocreator/memory-size-experiment-for-rwkv/resolve/main/Echo-A-1B5-Stage2.pth
Resolving huggingface.co (huggingface.co)... 54.230.111.25, 54.230.111.86, 54.230.111.56, ...
Connecting to huggingface.co (huggingface.co)|54.230.111.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/cb/ef/cbef09abb2634a3375b28868bffa285226dfeabedec89b28c2fb302221164d66/2aa6bafcab9f4d4cca59f4a2960805ac8fdd32e15f3fe08238df728aef1c093e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27Echo-A-1B5-Stage2.pth%3B+filename%3D%22Echo-A-1B5-Stage2.pth%22%3B&Expires=1689418097&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTY4OTQxODA5N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9jYi9lZi9jYmVmMDlhYmIyNjM0YTMzNzViMjg4NjhiZmZhMjg1MjI2ZGZlYWJlZGVjODliMjhjMmZiMzAyMjIxMTY0ZDY2LzJhYTZiYWZjYWI5ZjRkNGNjYTU5ZjRhMjk2MDgwNWFjOGZkZDMy

## Prepare the dataset

Prepare and preload the finetuning process dataset

In [2]:
%%script bash
# Reset the dataset dir
mkdir -p ./dataset
rm -rf ./dataset/*.jsonl

# Generate the various datasets
echo "## Generating word reptition dataset ##"

# We do a strong bias for smaller word count, to teach the concept from scratch
# so that the model can learn the function
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-2-count.jsonl  2  20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-5-count.jsonl  5  20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-10-count.jsonl 10 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-20-count.jsonl 20 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-40-count.jsonl 40 20000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-80-count.jsonl 80 20000 &

# With a slight mix of the larger word count
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-100-count.jsonl 100 5000 &
python ./memory_script/gen_limited_segmented_jsonl.py ./dataset/word-200-count.jsonl 100 5000 &

wait
echo "## Done ##"

ls -alh ./dataset/

## Generating word reptition dataset ##
Generated JSONL file with - 2 max words, 20000 samples - at ./dataset/word-2-count.jsonl
Generated JSONL file with - 5 max words, 20000 samples - at ./dataset/word-5-count.jsonl
Generated JSONL file with - 100 max words, 5000 samples - at ./dataset/word-200-count.jsonl
Generated JSONL file with - 100 max words, 5000 samples - at ./dataset/word-100-count.jsonl
Generated JSONL file with - 10 max words, 20000 samples - at ./dataset/word-10-count.jsonl
Generated JSONL file with - 20 max words, 20000 samples - at ./dataset/word-20-count.jsonl
Generated JSONL file with - 40 max words, 20000 samples - at ./dataset/word-40-count.jsonl
Generated JSONL file with - 80 max words, 20000 samples - at ./dataset/word-80-count.jsonl
## Done ##


## Configure your environment settings
(!Important: you will need to rerun the below cell, if you restart your kernel)

In [1]:
DEEPSPEED_STRAT="deepspeed_stage_1"
GPU_DEVICES="auto"
ENABLE_WANDB=True
WANDB_PREFIX="Echo-A-1B5"

print("DEEPSPEED_STRAT:", DEEPSPEED_STRAT)
print("ENABLE_WANDB:", ENABLE_WANDB)
print("GPU_DEVICES:", GPU_DEVICES)

if ENABLE_WANDB:
    WANDB_MODE="online"
else:
    WANDB_MODE="disabled"

# Computing the notebook, and various paths
import os
NOTEBOOK_DIR=os.path.dirname(os.path.abspath("__file__"))
PROJECT_DIR=os.path.abspath(os.path.join(NOTEBOOK_DIR, "../../../"))
TRAINER_DIR=os.path.abspath(os.path.join(PROJECT_DIR, "./RWKV-v4neo/"))

print("NOTEBOOK_DIR:", NOTEBOOK_DIR)
print("TRAINER_DIR:", TRAINER_DIR)
print("PROJECT_DIR:", PROJECT_DIR)

DEEPSPEED_STRAT: deepspeed_stage_1
ENABLE_WANDB: True
GPU_DEVICES: auto
NOTEBOOK_DIR: /root/picocreator-memory-experiment/notebook/experiment/memory
TRAINER_DIR: /root/picocreator-memory-experiment/RWKV-v4neo
PROJECT_DIR: /root/picocreator-memory-experiment


## Stage 3 : Simple Memory finetuning

In [4]:
# Lets preload the requried dataset (enwiki_100k)
!cd "{TRAINER_DIR}" && \
    python3 preload_datapath.py "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-1.yaml"

# Ensure the checkpoint directory exists
!cd "{TRAINER_DIR}" && mkdir -p "../checkpoint/Echo-A-1B5-mem-finetune-1/"

Extracting data files: 100%|█████████████████████| 1/1 [00:00<00:00, 510.50it/s]
Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-e8213534230c3c63/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.
100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 195.02it/s]


In [2]:
# Start the foundation model training
!cd "{TRAINER_DIR}" && \
    export WANDB_MODE="{WANDB_MODE}" && \
    python lightning_trainer.py fit \
        -c "{NOTEBOOK_DIR}/Echo-A-1B5-mem-finetune-1.yaml" \
        --trainer.logger.init_args.name="{WANDB_PREFIX} - Mem-Finetune-1 (bs=256, train-ctx=512, {DEEPSPEED_STRAT})" \
        --trainer.strategy="{DEEPSPEED_STRAT}" \
        --trainer.devices="{GPU_DEVICES}"  \
        --model.ctx_len=512

Setting ds_accelerator to cuda (auto detect)
[RWKV.model] Running RWKV model using 'torch-jit' with torch '2.0.1+cu118'
/usr/local/lib/python3.11/dist-packages/lightning/fabric/utilities/seed.py:39: UserWarning: No seed found, seed set to 1099263640
  rank_zero_warn(f"No seed found, seed set to {seed}")
Global seed set to 1099263640
wandb: Currently logged in as: picocreator. Use `wandb login --relogin` to force relogin
wandb: wandb version 0.15.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Tracking run with wandb version 0.15.4
wandb: Run data is saved locally in ./wandb/run-20230712_105219-pv2bcx8c
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run (8x3090) Echo-A-1B5 - Mem-Finetune-1 (bs=256, train-ctx=512, deepspeed_stage_1)
wandb: ⭐️ View project at https://wandb.ai/picocreator/RWKV-Memory-Experiment
wandb: 🚀 View run at https://wandb.ai/picocreator/RWKV-Memory-Experiment/runs/pv2bcx8c
Using /root/.cache/torch_extensions/py31

In [4]:
# Lets export the model from the checkpoint
!cd "{TRAINER_DIR}" && \
    python export_checkpoint.py \
        "../checkpoint/Echo-A-1B5-mem-finetune-1/last.ckpt" \
        "../model/Echo-A-1B5-Tune1.pth"
!cd "{TRAINER_DIR}" && ls -alh ../model/Echo-A-1B5-Tune1.pth

Setting ds_accelerator to cuda (auto detect)
Processing zero checkpoint '../checkpoint/Echo-A-1B5-mem-finetune-1/last.ckpt/checkpoint'
Detected checkpoint of type zero stage ZeroStageEnum.optimizer_states, world_size: 8
Parsing checkpoint created by deepspeed==0.9.3
Reconstructed fp32 state dict with 438 params 1515106304 elements
Saving fp32 state dict to ../model/Echo-A-1B5-Tune1.pth


In [5]:
# Lets do a quick dragon prompt validation
!python3 ./memory_script/eval_memory_guided.py "{PROJECT_DIR}/model/Echo-A-1B5-Tune1.pth"

Using /root/.cache/torch_extensions/py311_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py311_cu118/wkv_cuda...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py311_cu118/wkv_cuda/build.ninja...
Building extension module wkv_cuda...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
[1/3] c++ -MMD -MF wrapper.o.d -DTORCH_EXTENSION_NAME=wkv_cuda -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /usr/local/lib/python3.11/dist-packages/torch/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/torch/csrc/api/include -isystem /usr/local/lib/python3.11/dist-packages/torch/include/TH -isystem /usr/local/lib/python3.11/dist-packages/torch/include/THC -isystem /usr/local/cuda/include -isystem /usr/include/python3.11 -D_GLIBCXX